# 3. Models included in Progpy
ProgPy includes a subpackage with prognostic models developed by ProgPy contributors. These models can be configured, used, and modified by users for their purposes. All models are part of the models subpackage (`progpy.models`). Each model is different, and model documentation can be found [here](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html).

In this section we will briefly go over the specifics of each model, and how it could be used in simulation.

## ThrownObject

ThrownObject is typically the first model individuals are introduced to when learning to use progpy. This is because it's so simple. ThrownObject is a model of an object being thrown directly into the air and returning to the earth. The object is affected by gravity and air resistence in its path. The model includes two events: 1. `falling`- when the object is falling towards the ground, and 2. `impact`- when the object has impacted the ground.

This model has been used extensively in the previous sections. Rather than repeating this here, I direct the user to the sections on Simulating to Threshold a Parameters in 1. Simulating with Prognostics Models.

## Battery Models

The battery models are the most mature and widely used models in the ProgPy package. And there are a lot of them, each serving a different purpose and simulating different phenomenon. We will introduce each of them in this example, describing how to select, configure, and use each.

There are two ways to divide the models included in progpy, the first is by what they're simulating: *discharge* or *degredation*. 

*Discharge* is the process of using the batteries energy. When a battery is fully discharged, it no longer can be used for power. This point is typically called End of Discharge (EOD). 

*Degredation* is the gradual decrease of performance of a battery with use. This in easiest to see in the decrease in capacity, but it effects other parameters as well. 

The second dimension of classification is by model strategy: Equivalent Circuit or Electrochemistry. Equivalent circuit models represent the behavior of the model as an electrical circuit, while electrochemistry models model the chemical phenomenon at play internal to the battery. Electrochemistry models are typically more accurate, but they also are more complex, and therefore slower to run.

The classification of the six models into those categories are illustrated below:
| | Equivalent Circuit | Electrochemistry | 
| --- | --- | --- |
| Discharge | [BatteryCircuit](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryCircuit) | [BatteryElectroChemEOD](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryElectroChemEOD) |
| Degredation | | [BatteryElectroChemEOL](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryElectroChemEOL) | 
| Both |  | [BatteryElectroChemEODEOL](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryElectroChemEODEOL)  |

These models are described in more detail below:

### BatteryCircuit

`progpy.models.BatteryCircuit` is a vectorized prognostics model of a battery discharge, where it's behavior is represented by an equivalent circuit, below. The specifics of the model are described in the papaer [*Advanced Methods for Determining Prediction Uncertainty in Model-Based Prognostics with Application to Planetary Rovers*](https://papers.phmsociety.org/index.php/phmconf/article/view/2253).

![Equivalent Circuit](figs/equivcircuit.png)

This is the simplest of the battery models, and as a result, the least computationally expensive. This is also the least accurate or complete of the models. It is a simple approximation of the behavior of a battery during discharge. When deciding wether to use equivalent circuit or electrochemistry model, users should consider what accuracy they need, and what computational resources they have available.

First, we will import and instantiate the `BatteryCircuit` model to describe it further. 

In [ ]:
from progpy.models import BatteryCircuit
m = BatteryCircuit()

Battery circuit predicts one [event](https://nasa.github.io/progpy/glossary.html#term-event): End of Discharge, denoted as `EOD`.

In [ ]:
print(m.events)

`EOD` is the point at which the battery no longer has sufficient charge to be used.

The battery [outputs](https://nasa.github.io/progpy/glossary.html#term-output) (i.e, measurable parameters) are voltage and temperature, denoted by `v` and `t`, respectively.

In [ ]:
print(m.outputs)

Its one [input](https://nasa.github.io/progpy/glossary.html#term-input) (i.e., how it is loaded) is the current drawn from the battery, denoted by `i`.

In [ ]:
print(m.inputs)

The `BatteryCircuit` model has four states: temperature (`tb`), and the charge stored in different capacitors in the model (`qb`, `qcp`, `qcs`). For more information on these states, see the paper at the top of this section.

In [ ]:
print(m.states)

The batteries parameters are listed below. Each of these describe different aspects of the circuit. For details on these parameters, see the paper above, or the model documentation [here](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryCircuit).

In [ ]:
print(m.parameters)

Generally, for a complex model like this these parameters are not set directly, instead they are estimated using parameter estimation (see 2. Parameter Estimation). VEOD, however, must be set manually. VEOD is the voltage at which the battery no longer can perform its function.

For this model it's also a good idea to re-estimate parameters regularly. The parameters shift as the battery ages, especially qMax (the battery capacity).

To get to know this model better, let's use it in simulation. First we have to define a loading (i.e., current) profile for simulation. In this example we will start with a constant load. See the future load section in 01_Simulation for more details on future loading.

In [ ]:
load = m.InputContainer({'i': 2.5})
def future_load(t, x=None):
    return load

Next, let's use this loading profile to simulate until End of Discharge (EOD). EOD occurs when the voltage of the battery reaches the value of the parameter VEOD. We'll generate plots of the outputs (temperature and voltage)

In [ ]:
results = m.simulate_to_threshold(future_load, dt=('auto', 1), save_freq=1)
fig = results.outputs.plot(compact=False)

The first thing to notice here is that voltage (bottom graph) decreases with time. At the beginning of the discharge, voltage drops suddenly, this is from the first application of current. Then voltage decreases fairly linerally with time, until it hits a knee point, just before EOD. In general, this is the voltage curve you would expect from a battery during discharge.

The EOD event state decreases linerally with time, as seen below:

In [ ]:
fig = results.event_states.plot()

Remember that this in this case a constant load (i.e., current) was applied to the battery. Let's repeat this with a more complex load profile. In this case we'll use a piecewise profile, defined below.

In [ ]:
from progpy.loading import Piecewise
future_load = Piecewise(
    m.InputContainer,
    [600, 900, 1800, 3600],
    {'i': [2, 1, 4, 2, 3]})

results = m.simulate_to_threshold(future_load, dt=('auto', 1), save_freq=1)
fig = results.inputs.plot()
fig = results.outputs.plot(compact=False)

Here we see voltage following a similar curve from before, but there are jumps as the current changes. Voltage is directly correlated with the current draw. When the current draw decreases, voltage will increase, and vice versa.

There is some sensitivity to timestep in capturing  behavior, and for a dt above about 2 seconds, it doesn't work at all.

In [ ]:
results = m.simulate_to_threshold(future_load, dt=('auto', 2), save_freq=2)
fig = results.outputs.plot(compact=False, title="dt = 2")

results = m.simulate_to_threshold(future_load, dt=('auto', 2.5), save_freq=2.5)
fig = results.outputs.plot(compact=False, title="dt = 2.5")

This instability is very dependent on model parameters, for a differntly configured model it might be stable for a different range of dts.

### BatteryElectroChemEOD

Similar to BatteryCircuit, `progpy.models.BatteryElectroChemEOD` is a vectorized prognostics model of a battery discharge. Except the behavior is modeled not as a circuit, but instead by the electrochemical behavior of a battery.

The specifics of the model are described in the papaer [*Electrochemistry-based Battery Modeling for Prognostics*](https://papers.phmsociety.org/index.php/phmconf/article/view/2252).

First, we will import and instantiate the `BatteryElectroChemEOD` model to describe it further. 

In [ ]:
from progpy.models import BatteryElectroChemEOD
m = BatteryElectroChemEOD()

The events, outputs, and inputs are identical to `BatteryCircuit` above

In [ ]:
print('events', m.events)
print('inputs', m.inputs)
print('outputs', m.outputs)

The `BatteryElectroChemEOD` model has eight states. For more information on these states, see the paper at the top of this section.

In [ ]:
print(m.states)

The batteries parameters are listed below. Each of these describe different aspects of the circuit. For details on these parameters, see the paper above, or the model documentation [here](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryElectroChemEOD).

In [ ]:
print(m.parameters)

Note that there are significantly more parameters and twice as many states in `BatteryElectroChemEOD` when compared to `BatteryCircuit`. The increased complexity allows the model to better represent battery discharge behavior, especially thermal effects which are not very well represented in the `BatteryCircuit` model.

Generally, for a complex model like this these parameters are not set directly, instead they are estimated using parameter estimation (see 2. Parameter Estimation). VEOD, however, must be set manually. VEOD is the voltage at which the battery no longer can perform its function.

Like with `BatteryCircuit`, for this model it's also a good idea to re-estimate parameters regularly. The parameters shift as the battery ages, especially qMobile, Ro, and tDiffusion.

To get to know this model better, let's use it in simulation. First we have to define a loading (i.e., current) profile for simulation. In this example we will start with a constant load. See the future load section in 01_Simulation for more details on future loading.

In [ ]:
load = m.InputContainer({'i': 2.5})
def future_load(t, x=None):
    return load

results = m.simulate_to_threshold(future_load, dt=('auto', 1), save_freq=1)
fig = results.outputs.plot(compact=False)

Note here the same characteristic voltage profile shape (bottom). The crver are not exactly the same. Note the end of discharge occurs 300 seconds earlier in this model than in the batterycircuit. Also, the temperature graph has a different shape. This is where you can see the better performance in the `BatteryElectroChemEOD` model. 

Now let's repeat this with the piecewise load profile.

In [ ]:
from progpy.loading import Piecewise
future_load = Piecewise(
    m.InputContainer,
    [600, 900, 1800, 3600],
    {'i': [2, 1, 4, 2, 3]})

results = m.simulate_to_threshold(future_load, dt=('auto', 1), save_freq=1)
fig = results.inputs.plot()
fig = results.outputs.plot(compact=False)

Here we see voltage following a similar curve from before, but there are jumps as the current changes. Voltage is directly correlated with the current draw. When the current draw decreases, voltage will increase, and vice versa.

Like with `BatteryCircuit`, there is some sensitivity to timestep in capturing  behavior, and higher dt's will result in unstable and inaccurate behavior.

In [ ]:
results = m.simulate_to_threshold(future_load, dt=('auto', 3), save_freq=3)
fig = results.outputs.plot(compact=False, title="dt = 3")

results = m.simulate_to_threshold(future_load, dt=('auto', 10), save_freq=10)
fig = results.outputs.plot(compact=False, title="dt = 10")

results = m.simulate_to_threshold(future_load, dt=('auto', 12), save_freq=12)
fig = results.outputs.plot(compact=False, title="dt = 12")

### BatteryElectroChemEOL

### BatteryElectroChemEODEOL

## Centrifugal Pump

## Pneumatic Valve

## Electric Powertrain

## Aircraft Flight Models

## Conclusions